In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
# from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
# import torch
# import transformers

# from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

import os
import sys


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
directory_path = "./wikitextForSlavesOnWikiandEnslaved"

In [4]:
files = os.listdir(directory_path)

In [5]:
allmodule_file_path = "./ModulesR/WB_Schema_Relationships.txt"
with open(allmodule_file_path, "r", encoding="utf-8") as module_file:
        module_content = module_file.read()

In [6]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    # cache_dir="/data/yash/base_models",
    device_map='auto'
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sanazsn\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                          # cache_dir="/data/yash/base_models"
                                         )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from langchain import HuggingFacePipeline
from transformers import pipeline
import torch
pipe=pipeline("text-generation",
              model=llm,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30

              )

In [16]:
llm1 =HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [17]:

file = files[1]
if file.endswith('.txt'):
    document=[]
    dir_path = os.path.join(directory_path, file)
    text_path = dir_path
    loader=TextLoader(text_path, encoding = 'UTF-8')
    document.extend(loader.load())
    document_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])
    document_chunks=document_splitter.split_documents(document)
    # embeddings = OpenAIEmbeddings()
    # embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
    vectordb.persist()
    # llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
    pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm1,
                                              retriever=vectordb.as_retriever(search_kwargs={'k':10}),
                                              verbose=False, memory=memory,return_source_documents=True)
    # for j in module_content1:
    result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module, modules:"+str(module_content)+ "and populate the ontology based on the text in the form of the example format:Example format: Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file."})
    summ = result['answer']

    source_documents = result.get("source_documents", [])
    sen=""
    for doc in source_documents:
        sen = sen + str(doc.page_content)
        sen = sen + "\n"
    f_name = os.path.join("./wikitextForSlavesOnWikiandEnslaved/responses/llama3_WB_AllModules",os.path.splitext(file)[0])
    if not os.path.exists(f_name):
          os.makedirs(f_name, exist_ok=True)
    # response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
    # response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
    response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
    response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
    with open(response_file_path1, "w", encoding="utf-8") as response_file1:
        response_file1.write(summ)
    with open(response_file_path, "w", encoding="utf-8") as response_file:
        response_file.write(sen)
    # torch.cuda.empty_cache()
    
    del result
    del summ
    del source_documents
    del sen
    memory.clear()

C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
C:\Users\sanazsn\Anaconda3\envs\rag\Lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.44 GiB. GPU 

In [ ]:
module_content1={}
for f in modules_files:
  if f.endswith(".txt"):
    m_path1 = m_path+f

    with open(m_path1, "r", encoding="utf-8") as module_file:
       file_name = f.replace(".txt", "")
       module_content1[file_name] = module_file.read()

In [ ]:

file = files[1]
if file.endswith('.txt'):
    document=[]
    dir_path = os.path.join(directory_path, file)
    text_path = dir_path
    loader=TextLoader(text_path, encoding = 'UTF-8')
    document.extend(loader.load())
    document_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])
    document_chunks=document_splitter.split_documents(document)
    # embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
    vectordb.persist()
    # llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True,output_key='answer')
    pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                              retriever=vectordb.as_retriever(search_kwargs={'k':1}),
                                              verbose=False, memory=memory,return_source_documents=True)
    for j in module_content1:
        result=pdf_qa({"question":"Retrieve the related parts of the document that match with this ontology module, modules:"+str(module_content1[j])+ "and populate the ontology based on the text in the form of the example format:Example format: Name Record Module:1. has_Name(Agent, Name): has_Name(Joseph Vance Lewis, 'Joseph Vance Lewis') 2. has_Surname(Agent, Surname): has_Surname(Joseph Vance Lewis, 'Lewis') 3. has_First_Name(Agent, First_Name): has_First_Name(Joseph Vance Lewis, 'Joseph').Interagent Relationship Record Module: 1. has_Interagent_Relationship_Type_To(Agent, Relationship_Type): has_Interagent_Relationship_Type_To(Joseph Vance Lewis, 'Enslaver') 2. is_Relationship_To(Agent, Agent): is_Relationship_To(Joseph Vance Lewis, 'Bwril Nate'). Note: The Interagent Relationship Record Module describes the relationship to another agent who is an enslaver or owner. So use the keywords Enslaver or Owner to describe the relationship type. Sex Record Module: 1. has_Sex(Agent, Sex_Type): has_Sex(Joseph Vance Lewis, 'Male'). Occupation Record Module: 1. has_Occupation(Agent, Occupation): has_Occupation(Joseph Vance Lewis, 'Slave, Educator, Lawyer, and Autobiographer'). Age Record Module: 1. has_Age(Agent, Age): has_Age(Joseph Vance Lewis, 72) 2. has_BirthDate(Agent, Date_of_Birth): has_BirthDate(Joseph Vance Lewis, 'December 25, 1853') 3. has_DeathDate(Agent, Date_of_Death): has_DeathDate(Joseph Vance Lewis, 'April 24, 1925'). Person Status Record Module: 1. has_Person_Status (Agent, Person_Status): has_Person_Status(Joseph Vance Lewis, 'Enslaved Person, Lawyer') 2. has_Status_Generating_Event(Person_Status, Event): has_Status_Generating_Event(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), has_Status_Generating_Event(Lawyer, 'Emancipation') 3. recorded_At(Person_Status_information, Event): recorded_At(Enslaved Person, 'Sale of estate of Joseph Dubreuil'), recorded_At(Lawyer, 'Emancipation'). Race Record Module: has_Race_or_Color(Agent, Race_or_Color): has_Race_or_Color(Joseph Vance Lewis, African-American).  Note: Skip the relations for which there is no information listed in the text file."})
        summ1 = result['answer']
    
        source_documents = result.get("source_documents", [])
        sen=""
        for doc in source_documents:
            sen = sen + str(doc.page_content)
            sen = sen + "\n"
        f_name = os.path.join("./wikitextForSlavesOnWikiandEnslaved/responses/llama_WB_ModulesByModules",os.path.splitext(file)[0])
        if not os.path.exists(f_name):
              os.makedirs(f_name, exist_ok=True)
        response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_retrieved.txt")
        response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_"f"{j}_triples.txt")
        # response_file_path = os.path.join(f_name, f"{os.path.splitext(file)[0]}_retrieved.txt")
        # response_file_path1 = os.path.join(f_name, f"{os.path.splitext(file)[0]}_triples.txt")
        with open(response_file_path1, "w", encoding="utf-8") as response_file1:
            response_file1.write(summ1)
        with open(response_file_path, "w", encoding="utf-8") as response_file:
            response_file.write(sen)
        # torch.cuda.empty_cache()
        
        del result
        del summ1
        del source_documents
        del sen
        memory.clear()